In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
today = datetime.today()
# Библиотеки для моделирования
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# Стоп-слова
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oigla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# stop_words

Для модели был специально подготовлен датасет, в котором только 59013 строк, 12 полей.
Метод подготовки датасета:

plan_99 = plan.sample(frac=0.99)
plan = plan.drop(plan_99.index)

In [4]:
plan = pd.read_csv('I:/Download/Датасеты/test_for_train_model.csv')
test = pd.read_csv('I:/Download/Датасеты/test.csv')

TF-IDF Vectorizer - векторизируем наш датафрейм, а затем расчитав косинусное расстояние между векторами, подберем группы для наших участников

In [5]:
tfidf = TfidfVectorizer(stop_words=list(stop_words))
plan['type3'] = plan['type3'].fillna('')
overview_matrix = tfidf.fit_transform(plan['type3'])
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
mapping = pd.Series(plan.index,index =plan['type2'])
print(mapping)

type2
ОНЛАЙН Красота и стиль           0
ОНЛАЙН Домоводство               1
ОНЛАЙН Домоводство               2
ОНЛАЙН Домоводство               3
ОНЛАЙН Домоводство               4
                             ...  
Информационные технологии    59008
Иностранные языки            59009
Иностранные языки            59010
Спортивные игры              59011
Информационные технологии    59012
Length: 59013, dtype: int64


In [6]:
plan.index

RangeIndex(start=0, stop=59013, step=1)

In [7]:
overview_matrix

<59013x458 sparse matrix of type '<class 'numpy.float64'>'
	with 178006 stored elements in Compressed Sparse Row format>

In [8]:
similarity_matrix

array([[1.       , 0.0271926, 0.0271926, ..., 0.       , 0.       ,
        0.       ],
       [0.0271926, 1.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       [0.0271926, 1.       , 1.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 1.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 1.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        1.       ]])

In [19]:
def recommend_plan(plan_input):
    plan_index = mapping[plan_input]
    similarity_score = list(enumerate(similarity_matrix[plan_index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[0], reverse=True)
    similarity_score = similarity_score[1:10]
    plan_indices = [i[0] for i in similarity_score]
    return (plan['numgroup'].iloc[plan_indices])

In [20]:
print(recommend_plan('ОНЛАЙН Гимнастика'))

8322    801350520
8321    801349601
8320    801369467
8319    801366648
8318    801373454
8317    801353771
8316    801352950
8315    801351464
8314    801347511
Name: numgroup, dtype: int64


In [21]:
# результат сохраняем в таблицу test
test['уникальный номер группы'] = np.resize(list(recommend_plan('ОНЛАЙН Гимнастика')),len(test))
test.to_csv('I:/Download/Датасеты/test_result.csv')
test.head()

,уникальный номер участника,уникальный номер группы
0,101346610,801350520
1,101346612,801349601
2,101347269,801369467
3,101347321,801366648
4,101347983,801373454


Ниже разберем решение

In [12]:
# Посмотрим какие слова остались после фильтрации
print(tfidf.get_feature_names_out())

['android' 'ios' 'автомобильная' 'авторская' 'авторские' 'адаптивная'
 'академического' 'акварельная' 'аксессуаров' 'английская' 'английский'
 'андроид' 'анти' 'арабский' 'арифметика' 'архитектура' 'астрономия'
 'атласными' 'атласных' 'атлетическая' 'аэробика' 'бадминтон' 'базовый'
 'бальные' 'бардовская' 'баскетбол' 'батик' 'бесконфликтное' 'бизнес'
 'бизнесе' 'бильярд' 'бисероплетение' 'бодибалет' 'большой' 'бочче' 'боя'
 'бояться' 'бумаги' 'валяние' 'вводный' 'вечера' 'вещей' 'видеомонтажа'
 'видеостудия' 'витражное' 'внука' 'внукам' 'внуками' 'возрасте' 'вокруг'
 'волейбол' 'волосами' 'восточные' 'встречи' 'всё' 'вторая' 'выжить'
 'вышивка' 'вязание' 'генеалогия' 'география' 'гимнастика' 'гитаре'
 'гладью' 'глаз' 'глина' 'городки' 'грамотности' 'грамотность' 'графика'
 'графики' 'графических' 'греческий' 'гто' 'дартс' 'движение'
 'декоративных' 'декупаж' 'дереву' 'деятельность' 'джиу' 'джицу' 'дизайн'
 'дизайнер' 'дискуссионный' 'долголетия' 'домашних' 'дополнительного'
 'доступный

In [13]:
# Посмотрим IDF слов 
tfidf.idf_

array([10.88691769, 10.88691769,  7.51962186,  9.68294489,  6.6479919 ,
        6.31564906,  7.47467048,  6.56057997,  7.06554906,  9.68294489,
        4.20648134, 10.88691769,  6.68222507,  8.43018192,  6.11623307,
        7.53118269,  9.34647265,  7.89118542, 10.88691769, 11.2923828 ,
        7.84239526,  8.22432987,  6.85563127,  5.78502081, 10.88691769,
        9.90608844,  8.72743344, 11.2923828 ,  9.21294126, 10.37609207,
        6.72803461,  9.68294489,  9.21294126,  9.27747978,  9.90608844,
       10.88691769,  9.09515822,  9.50062333,  8.37461207, 11.2923828 ,
       10.59923562,  8.80747615,  9.78830541,  7.19803824, 10.03961983,
        7.16524842,  7.16524842, 10.03961983,  3.66070868,  5.9381578 ,
        7.16524842,  8.72743344,  6.42870192, 10.37609207,  7.60350335,
        8.80747615, 11.2923828 ,  7.69507054,  5.96693677,  7.75142348,
        5.9381578 ,  2.84926681,  8.72743344,  9.34647265,  6.37972792,
        7.76602228, 10.59923562,  5.16660592,  6.08563263,  7.27

In [14]:
# Посмотрим на количество документов и количество токенов (слов).
overview_matrix.shape

(59013, 458)

In [15]:
"""посмотрим на значение TF-IDF для конкретного слова в конкретном документе. 
Чем оно уникальнее для конкретного документа, тем выше показатель"""
df_tfidf = pd.DataFrame(overview_matrix.toarray(), columns = tfidf.get_feature_names_out())
print(df_tfidf.T)

               0      1      2      3      4      5      6      7      8      \
android          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
ios              0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
автомобильная    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
авторская        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
авторские        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...              ...    ...    ...    ...    ...    ...    ...    ...    ...   
это              0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
язык             0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
языки            0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
японские         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
японский         0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

               9      ...  59003  59004

In [16]:
# вычислим среднее арифметическое по строкам матрицы и преобразуем в список
mean_weights = np.asarray(overview_matrix.mean(axis = 0)).ravel().tolist()
print(mean_weights)

[1.3932573434911309e-05, 1.3932573434911309e-05, 0.0011218455473814219, 8.51583769883819e-05, 0.0022262415488366757, 0.0032744454336889432, 0.0007989606017557589, 0.002851129072744505, 0.001183972834866232, 0.00012384558366074696, 0.02596559920857862, 1.1407246659542408e-05, 0.0019435039375233596, 0.0005238121906297503, 0.0041759547269376385, 0.0008854612248893802, 0.00021794384865429995, 0.0005792190258735832, 1.2809532577880374e-05, 1.64304721127414e-05, 0.0009150768441277341, 0.0007117075898530829, 0.0011945455369873042, 0.006858993413693978, 2.7246003903265467e-05, 0.00011861793164218051, 0.00024673174592361723, 1.1982220547786885e-05, 0.00014632068784063172, 3.028408689700635e-05, 0.003236574991950935, 0.00015250876925423207, 0.0002541812820903868, 0.0002076193543798064, 8.38755438345082e-05, 1.6159370250227296e-05, 0.00015432248684511937, 9.312789130278344e-05, 0.0004289329255564701, 1.0781807132547981e-05, 3.594666164336066e-05, 0.00022454789938778058, 6.499251025056556e-05, 0.0

In [23]:
# создадим датафрейм из словаря
mean_weights_df = pd.DataFrame({'term': tfidf.get_feature_names_out(), 
                                'mean_weights': mean_weights})
 
# сортируем по убыванию 10 слов с максимальным средним TF-IDF (частота слов, обратная частота документа)
mean_weights_df.sort_values(by = 'mean_weights', ascending = False)\
    .reset_index(drop = True).head(20)

,term,mean_weights
0,онлайн,0.140635
1,гимнастика,0.097982
2,язык,0.037911
3,танцы,0.036505
4,суставная,0.031183
5,офп,0.028805
6,уходу,0.028176
7,мастер,0.028176
8,зрелом,0.028176
9,возрасте,0.028176
